In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('./data/ma405_tick.csv')

data.head()

,date,volume,turnover,open_interest,last_price,bid_price,ask_price,bid_volume,ask_volume
0,2024-03-21 10:35:09,678635,1.742056e+09,911573.0,2555.0,2554.0,2555.0,908,491
1,2024-03-21 10:35:10,678641,1.742071e+09,911573.0,2554.0,2554.0,2555.0,950,653
2,2024-03-21 10:35:11,678666,1.742136e+09,911593.0,2554.0,2554.0,2555.0,1006,633
3,2024-03-21 10:35:12,678888,1.742705e+09,911700.0,2555.0,2554.0,2555.0,999,423
4,2024-03-21 10:35:13,678914,1.742772e+09,911724.0,2555.0,2554.0,2555.0,1010,308


In [3]:
data.tail()

,date,volume,turnover,open_interest,last_price,bid_price,ask_price,bid_volume,ask_volume
468447,2024-05-17 14:59:52,0,0.0,1168.0,2755.0,2684.0,2755.0,44,20
468448,2024-05-17 14:59:56,0,0.0,1168.0,2755.0,2685.0,2755.0,11,20
468449,2024-05-17 14:59:57,0,0.0,1168.0,2755.0,2685.0,2755.0,44,20
468450,2024-05-17 14:59:58,0,0.0,1168.0,2755.0,2684.0,2755.0,44,20
468451,2024-05-17 15:00:00,0,0.0,1168.0,2755.0,2684.0,2755.0,44,20


In [4]:
mul = 10

data['volume_delta'] = data['volume'].diff()
data['turn_delta'] = data['turnover'].diff()
data['trade_price'] = data['turn_delta']/data['volume_delta']

In [5]:
def argmax(x):
    return x.argmax()
def argmin(x):
    return x.argmin()
def get_diff(x):
    if abs(x[0])>abs(x[1]):
        return x[0]
    else:
        return x[1]

In [6]:
p0 = 60
p1 = 360
p2 = 360


roll_0 = data['last_price'].rolling(p0)
roll_1 = data['last_price'].rolling(p1)
roll_2 = data['last_price'].rolling(p2)


max0 = roll_0.max()
min0 = roll_0.min()

max1 = roll_1.max()
min1 = roll_1.min()

max2 = roll_2.max().shift(-p2)
min2 = roll_2.min().shift(-p2)


data['min0_diff'] = data['last_price'] - max0
data['max0_diff'] = data['last_price'] - min0
data['min1_diff'] = data['last_price'] - max1 
data['max1_diff'] = data['last_price'] - min1
data['max2_diff'] = max2 - data['last_price']
data['min2_diff'] = min2 - data['last_price']

data['diff0'] = data[['min0_diff','max0_diff']].apply(get_diff,axis=1)
data['diff1']= data[['min1_diff','max1_diff']].apply(get_diff,axis=1)
data['diff2']=data[['min2_diff','max2_diff']].apply(get_diff,axis=1)


# sma = talib.SMA(data['last_price'],3)
# data['slope'] = talib.LINEARREG_SLOPE(data['last_price'],3)

# ask_sma_volume = talib.SMA(data['ask_volume'],3)
# bid_sma_volume = talib.SMA(data['bid_volume'],3)

# data['ask_volume_slope'] = talib.LINEARREG_SLOPE(ask_sma_volume,3)
# data['bid_volume_slope'] = talib.LINEARREG_SLOPE(bid_sma_volume,3)

In [15]:
dt = data[:-20000]
dt['diff2'].describe()

count    448452.000000
mean          0.218884
std           5.161285
min         -30.000000
25%          -3.000000
50%           2.000000
75%           4.000000
max          41.000000
Name: diff2, dtype: float64

In [78]:
dt[(dt['last_price']<=dt['bid_price'])&(dt['diff0']>0)&(dt['bid_volume']>2*dt['ask_volume'])]['max2_diff'].describe()

count    6753.000000
mean        3.996594
std         3.421533
min        -1.000000
25%         2.000000
50%         3.000000
75%         5.000000
max        37.000000
Name: max2_diff, dtype: float64

In [77]:
dt[(dt['last_price']>=dt['ask_price'])&(dt['diff0']<0)&(dt['ask_volume']>2*dt['bid_volume'])]['min2_diff'].describe()

count    2975.000000
mean       -4.719328
std         4.060266
min       -28.000000
25%        -6.000000
50%        -3.000000
75%        -2.000000
max         1.000000
Name: min2_diff, dtype: float64